The code is designed to be executed within [Docker](https://github.com/nazarb/2025_levees_DL/blob/main/Setup/Setup.md).


In [7]:
# The code allows for converting levees (line feature class) into raster data, depending on the width of the levee
# Raw data
# Buławka, Nazarij, and Hector A. Orengo. 2025. ‘The Dataset of Ancient Irrigation Traces in Central Iraq Digitised Using Multi-Temporal and Multi-Source Satellite Imagery’. Dane Badawcze UW. https://doi.org/10.58132/MGOHM8.
#
# Import the libraries
import geopandas as gpd
import rasterio
import os
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# The procedure consists of several steps, but it is pretty simple: 1. Read the data; 2. Calculate buffer values: multiplication of the buffer value (200 m) with a Size value (0.25, 0.5, 0.75); 3. Create shapes and rasterise them.

## Load the raw data and store the name of it
shapefile_path = input("Enter path to reference data: ").strip('"')
output_name = os.path.splitext(os.path.basename(shapefile_path))[0] + ".tif"
## Specify the location of the output. The original name will be included and saved with tif extension
output_path_ = input("Enter path to output data: ").strip('"')
output_path = os.path.join(output_path_, output_name)
os.makedirs(output_path_, exist_ok=True)  # ensure the folder exists

## Put a value, which will be stored in the Raster // For binary classification, this value is 1 
manual_type = int(input("Enter the 'Type' value to use for rasterisation (integer): "))
## Read the data
gdf = gpd.read_file(shapefile_path)
# Check if they are correct
if 'Size' not in gdf.columns:
    raise ValueError("Shapefile must contain the 'Size' column")

# Specify the buffer value, which will be used to calculate the width of the features.
gdf['Buffer'] = gdf['Size'] * 200 # replace with any value

# Filter out data that are incorrect and select levees ( Type = 2)
gdf = gdf[(gdf["Type"] == 2) & (gdf["geometry"].notnull())]

# Apply the buffer around each line
gdf['geometry'] = gdf.apply(lambda row: row.geometry.buffer(row['Buffer']), axis=1)

# Define the output metadata
minx, miny, maxx, maxy = gdf.total_bounds
resolution = 10  # Set spatial resolution
width = int((maxx - minx) / resolution)
height = int((maxy - miny) / resolution)
transform = from_bounds(minx, miny, maxx, maxy, width, height)

# Rasterization parametres
## Read the shapes
shapes = ((geom, manual_type) for geom in gdf['geometry'])
rasterized_levee = rasterize(
    shapes=shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,  # Use 0 to fill Nodata
    dtype='int32'  # The datatype for raster
)

# Save the raster
with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=height,
    width=width,
    count=1,
    dtype=rasterized_levee.dtype,
    crs=gdf.crs,  # Use the same coordinate system
    transform=transform
) as dst:
    dst.write(rasterized_levee, 1)

print(f"Buffered levees saved to '{output_path}'")


Enter path to reference data:  "I:\UnderTheSands\Results\UnderTheSANDS_CFE1_levees_1_02\Vector\UnderTheSands_Iraq_CFE_Network_of_levees_and_palaeo_channels.shp"
Enter path to output data:  "I:\UnderTheSands\Results\"
Enter the 'Type' value to use for rasterisation (integer):  1


Buffered levees saved to 'I:\UnderTheSands\Results\UnderTheSands_Iraq_CFE_Network_of_levees_and_palaeo_channels.tif'


In [9]:
    !python --version


Python 3.12.7
